# Part III - Uncertainty
## Project 3b - Heredity

[Course Link](https://cs50.harvard.edu/ai/)

[Project Instructions](https://cs50.harvard.edu/ai/projects/2/heredity/)

## Instructions
Write an AI to assess the likelihood that a person will have a particular genetic trait.

<img src='data/gene_network.png'>

Each person in the family has a Gene random variable representing how many copies of a particular gene (e.g., the hearing impairment version of GJB2) a person has: a value that is 0, 1, or 2. Each person in the family also has a Trait random variable, which is yes or no depending on whether that person expresses a trait (e.g., hearing impairment) based on that gene. There’s an arrow from each person’s Gene variable to their Trait variable to encode the idea that a person’s genes affect the probability that they have a particular trait. Meanwhile, there’s also an arrow from both the mother and father’s Gene random variable to their child’s Gene random variable: the child’s genes are dependent on the genes of their parents.

Your task in this project is to use this model to make inferences about a population. Given information about people, who their parents are, and whether they have a particular observable trait (e.g. hearing loss) caused by a given gene, your AI will infer the probability distribution for each person’s genes, as well as the probability distribution for whether any person will exhibit the trait in question.

**Ultimately, we’re looking to calculate these probabilities based on some evidence: given that we know certain people do or do not exhibit the trait, we’d like to determine these probabilities.**

**Recall from lecture that we can calculate a conditional probability by summing up all of the joint probabilities that satisfy the evidence, and then normalize those probabilities so that they each sum to 1.**

**Your task in this project is to implement three functions to do just that: joint_probability to compute a joint probability, update to add the newly computed joint probability to the existing probability distribution, and then normalize to ensure all probability distributions sum to 1 at the end.**


## Imports & Pre-Set Probabilties
**Note that the global variable PROBS conatins pre-set probabilties related to the gene for hearing loss, see instructions for more info**

In [1]:
import itertools
import sys
import pandas as pd

def load_file(f):
    df = pd.read_csv(f)
    df = df.where(pd.notnull, None)
    df = df.replace(0, False)
    df = df.replace(1, True)
    
    return df.set_index('name').T.to_dict()

# Probabilites of the GJB2 gene for hearing impairment in newborns
PROBS = {
    # Unconditional probabilities for having gene (parents both None)
    "gene": {
        2: 0.01,
        1: 0.03,
        0: 0.96
    },

    # Conditional probabilty that a person exhibits a trait
    # based on having (or not having) the gene for that trait
    "trait": {

        # Probability of trait given two copies of gene
        2: {
            True: 0.65,
            False: 0.35
        },

        # Probability of trait given one copy of gene
        1: {
            True: 0.56,
            False: 0.44
        },

        # Probability of trait given no gene
        0: {
            True: 0.01,
            False: 0.99
        }
    },

    # Mutation probability - probability that a gene mutates from
    # the gene in question to not being that gene, and vice-versa
    "mutation": 0.01
}


def powerset(s):
    """
    Return a list of all possible subsets of set s.
    """
    s = list(s)
    return [
        set(s) for s in itertools.chain.from_iterable(
            itertools.combinations(s, r) for r in range(len(s) + 1)
        )
    ]

## Family Data Sets
There are 3 family data sets containing the information needed to perform the heredity analysis. 

* trait volumn NaN values mean that we don't know if the person being examined 'name' has the trait or not

So for family 1:
* We know Harry's parents, but don't know if he has the trait
* For James, we don't know his parents, but he does exhibit the trait
* Lily, don't know her parents, doesn't exhibit trait

### Family 1

In [2]:
pd.read_csv('data/family0.csv')

,name,mother,father,trait
0,Harry,Lily,James,NaN
1,James,NaN,NaN,1.0
2,Lily,NaN,NaN,0.0


### Family 2

In [3]:
pd.read_csv('data/family1.csv')

,name,mother,father,trait
0,Arthur,NaN,NaN,0.0
1,Charlie,Molly,Arthur,0.0
2,Fred,Molly,Arthur,1.0
3,Ginny,Molly,Arthur,NaN
4,Molly,NaN,NaN,0.0
5,Ron,Molly,Arthur,NaN


### Family 3

In [4]:
pd.read_csv('data/family2.csv')

,name,mother,father,trait
0,Arthur,NaN,NaN,0.0
1,Hermione,NaN,NaN,0.0
2,Molly,NaN,NaN,NaN
3,Ron,Molly,Arthur,0.0
4,Rose,Hermione,Ron,1.0


In [5]:
def main(family):
    people = family
    
    # Keep track of gene and trait probabilities for each person
    probs = {
        person: {
            "gene": {
                2: 0,
                1: 0,
                0: 0
            },
            "trait": {
                True: 0,
                False: 0
            }
        }
        for person in people
    }
    
    # Loop over all sets of people who might have the trait
    names = set(people.keys())
    
    for have_trait in powerset(names):
        # Check if current set of people violates known information
        fails_evidence = any(
            (people[person]["trait"] is not None and
             people[person]["trait"] != (person in have_trait))
            for person in names
        )
        
        if fails_evidence:
            continue
            
        # Loop over all sets of people
        for one_gene in powerset(names):
            for two_genes in powerset(names - one_gene):
                # Update probabilities with new joint probability
                p = joint_probability(people, one_gene, two_genes, have_trait)
                update(probs, one_gene, two_genes, have_trait, p)
          
    # Converts probabilities to sum to 1
    normalize(probs)

In [6]:
def get_trait_prob(person, num_genes, has_trait):
    if person in has_trait:
        return PROBS["trait"][num_genes][True]
    else:
        return PROBS["trait"][num_genes][False]

    
def get_parent_gene(parent, child_genes, one_gene, two_genes):
    if parent in one_gene:
        return 0.50 

    if child_genes == 2:
        if parent in two_genes:
            return 1 - PROBS['mutation']
        else:
            return PROBS['mutation']
        
    if child_genes == 0:
        if parent in two_genes:
            return PROBS['mutation']
        else:
            return  1 - PROBS['mutation']
        
        
def joint_probability(fam, one_gene, two_genes, has_trait):
    joint_prob = 0
    
    for person, info in fam.items():
        trait = info['trait']
        
        # Parent Probabilty Calculation
        if info['mother'] == None and info['father'] == None:  
            gene_prob  = 0.0
            trait_prob = 0.0
            
            if person in one_gene:
                gene_prob = PROBS["gene"][1] 
                trait_prob = get_trait_prob(person, 1, has_trait)              
            elif person in two_genes:
                gene_prob = PROBS["gene"][2]
                trait_prob = get_trait_prob(person, 2, has_trait)   
            else:
                gene_prob = PROBS["gene"][0]
                trait_prob = get_trait_prob(person, 0, has_trait)
                
            # Update Joint Probabilty
            if joint_prob == 0:
                joint_prob = gene_prob * trait_prob
            else:
                joint_prob *= (gene_prob * trait_prob)
                          
        # Child Probability Calculation
        else:
            gene_prob  = 0.0
            trait_prob = 0.0
            child_prob = 0.0
            mother = info['mother']
            father = info['father']
            mother_prob = 0.0
            father_prob = 0.0
            
            # If child has one gene
            if person in one_gene:  
                from_mom  = 0.0
                not_mom   = 0.0
                from_dad = 0.0
                not_dad   = 0.0

                # mother prob
                if mother in one_gene:
                    from_mom = 0.50
                    not_mom = 0.50
                elif mother in two_genes:
                    from_mom = 1 - PROBS['mutation']
                    not_mom = 0.01
                else:
                    from_mom = 0.01
                    not_mom  = 1 - PROBS['mutation']
                                       
                # father prob
                if father in one_gene:
                    from_dad = 0.50
                    not_dad = 0.50
                elif father in two_genes:
                    from_dad = 1 - PROBS['mutation']
                    not_dad = 0.01
                else:
                    from_dad = 0.01
                    not_dad  = 1 - PROBS['mutation']
                    
                gene_prob = (from_mom * not_dad) + (from_dad * not_mom)
                trait_prob = get_trait_prob(person, 1, has_trait)          
              
            elif person in two_genes:                
                mother_prob = get_parent_gene(mother, 2, one_gene, two_genes)
                father_prob = get_parent_gene(father, 2, one_gene, two_genes)
                gene_prob = mother_prob * father_prob
                trait_prob = get_trait_prob(person, 2, has_trait)        
            else:             
                mother_prob = get_parent_gene(mother, 0, one_gene, two_genes)
                father_prob = get_parent_gene(father, 0, one_gene, two_genes)
                gene_prob = mother_prob * father_prob 
                trait_prob = get_trait_prob(person, 0, has_trait)            
            
            # Update Joint Probablity
            if joint_prob == 0:
                joint_prob = gene_prob * trait_prob
            else:
                joint_prob *= (gene_prob * trait_prob)
            
    return(joint_prob) 


def update(probs, one_gene, two_genes, have_trait, p):
    """
    Add to `probabilities` a new joint probability `p`.
    Each person should have their "gene" and "trait" distributions updated.
    Which value for each distribution is updated depends on whether
    the person is in `have_gene` and `have_trait`, respectively.
    """
    for person in probs:
        if person in one_gene:
            probs[person]['gene'][1] += p 
        elif person in two_genes: 
            probs[person]['gene'][2] += p 
        else:
            probs[person]['gene'][0] += p
            
        if person in have_trait:
            probs[person]['trait'][True] += p
        else:
            probs[person]['trait'][False] += p
        

def normalize(probs):
    '''
    For normalization factors, I used the formula: 1/sum(gene_results per person)
    '''
    # Normalize Genes
    for person, data in probs.items():
        gene_factor = 1 / sum(data['gene'].values()) 
        trait_factor = 1 / sum(data['trait'].values())
        
        print(f'{person}:')
        print(f'  Gene:')
        for gene, value in data['gene'].items():
            print(f'     {gene}: {value * gene_factor :.4f}')
            value *= (value * gene_factor)
            
            
        print(f'  Trait:')   
        for trait, value in data['trait'].items():
            print(f'     {gene}: {value * trait_factor :.4f}')
            value *= (value * trait_factor)
        
        print()

In [7]:
fam = load_file('data/family0.csv')
main(fam)

Harry:
  Gene:
     2: 0.0092
     1: 0.4557
     0: 0.5351
  Trait:
     0: 0.2665
     0: 0.7335

James:
  Gene:
     2: 0.1976
     1: 0.5106
     0: 0.2918
  Trait:
     0: 1.0000
     0: 0.0000

Lily:
  Gene:
     2: 0.0036
     1: 0.0136
     0: 0.9827
  Trait:
     0: 0.0000
     0: 1.0000



In [8]:
# Family 0 Child Harry's Genes Prior to Normalization
a = 0.000292184739
b = 0.014499220722
c = 0.017026184539

# Normalization Calculations
fac = (a+b+c)
t = 1/fac
tot = t * a + t * b + t * c

print(fac)
print(t)
print(tot)

print()
print(f'2 Genes: {a * t :.4f}')
print(f'1 Gene:  {b * t :.4f}')
print(f'0 Genes: {c * t :.4f}')

print()
print(a/fac)
print(b/fac)
print(c/fac)

0.03181759
31.429156010873232
1.0

2 Genes: 0.0092
1 Gene:  0.4557
0 Genes: 0.5351

0.009183119746027276
0.455698270107824
0.5351186101461487
